In [8]:
chave_openai = "<chave de api openai>"

In [9]:
import os
import pandas as pd
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [10]:
import textract
doc = textract.process("./manual_aspirador_eng.pdf")

with open('manual_aspirador_eng.txt', 'w', encoding="utf-8") as f:
    f.write(doc.decode('utf-8'))

with open('manual_aspirador_eng.txt', 'r', encoding="utf-8") as f:
    text = f.read()

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=chave_openai, model="text-embedding-ada-002")

db = FAISS.from_documents(chunks, embeddings)

In [12]:
query = "Robot does not work off the charging dock. Why?"
docs = db.similarity_search(query)
docs[0]

Document(lc_kwargs={'page_content': 'yourself.\n\n\n\n24\n\n\n\n\x0cInvestigation of common problems\n\nNo.\n\n\n\nPrompt Tone\n\n\n\nCause of Failure\n\n\n\nSolution\n\n\n\n1\n\n\n\nWheels off the ground.\n\n\n\nHost suspended.\n\n\n\nPlease put the host back to the ground and press the\n\ncleaning key again.\n\n\n\n2\n\n\n\nDust box not installed.\n\n\n\nDust box removed and not installed.\n\n\n\nReplace the dustbin to the host and press the cleaning\n\nbutton again.\n\n\n\n3\n\n\n\nBattery failure.\n\n\n\nAbnormal battery charging.\n\n\n\nContact after-sales service to test the battery.\n\n\n\n4\n\n\n\nPlease check if the wheel is stuck.\n\n\n\nWheel jammed.\n\n\n\nCheck whether the wheel is entangled with foreign objects\n\nand clean up the entangled objects.\n\n\n\n5\n\n\n\nPlease check if the front bumper is\n\nstuck.\n\n\n\nFront bumper bracket stuck.\n\n\n\nCheck whether the front bumper can be bounced\n\nnormally.\n\n\n\n6\n\n\n\nRobot connection failed.\n\n\n\nMachine failed 

In [13]:
chain = load_qa_chain(OpenAI(openai_api_key=chave_openai, temperature=0), chain_type="stuff")

chain.run(input_documents=docs, question=query)

' Check if the wheels are stuck or entangled with foreign objects. If so, clean up the entangled objects.'